## A hypothetical real (but very pessimistic) lower bound $\forall k$

$$V_l = \frac{\lambda}{1 + \sum_{j=0}^{\lambda-1} \sum_{i=1}^{\frac{n}{\lambda}(\lambda-j)} \frac{1}{i}}$$

where $d$ is the graph's diameter.

Let $u$ be the most advanced node in computation and suppose it being at iteration $t$. _Remember "node $u$ is at iteration $t$" means that $u$ has already performed the $t$-th update of its local model, so it owns $\mathbf{w}^{(t)}_u$_.

Let $d(u,v)$ be the "directed" distance from $u$ to $v$, e.g. the minimum amount of directed edges one needs to cross in order to go from $u$ to $v$ (obviously edges can be crossed just in the proper direction), then:

- nodes at distance 1 from $u$ must be at least at iteration $t-1$;
- nodes at distance 2 from $u$ must be at least at iteration $t-2$;
- in general, nodes at distance $l : l \leq d$ from $u$ must be at least at iteration $t-l$.

Due to exponential memoryless property, all exponential variables that rule the duration of each node's iteration can be regenerated in any moment. Therefore is legit assuming that right now node $u$ is starting iteration $t$ and nodes at distance $l$ from $u$ are starting iteration $t-l$ for all $l \leq d$.

The worst case for node $u$ is the following:
- by the time node $u$ finishes iteration $t$ it doesn't own inputs from its parent to start iteration $t+1$. So, node $u$ has finished, all random exponential variables can be regenerated, $u$ have to wait for the slowest node among those at distance 1 from it;
- but nodes at distance 1 from $u$ cannot start iteration $t$ since their still waiting their parents for their outcomes (e.g. nodes at distance 2 from $u$ haven't performed iteration $t-1$ yet); the same for nodes at distance 2 from $u$: they are waiting their parents, and so on up to nodes at distance $d$ from $u$, a straggler belongs to them, it is slowing down the whole computation.

In this case, the time taken by the slowest node at distance $d$ is distributed as $max \{X_1,...,X_n\}$ (where $X_i \sim Exp(\lambda)$ and $n$ is the total amount of computational units) since it is . After waiting this time a chain of calculations starts up to nodes at distance 1 from $u$, and after they have finished their computations, then $u$ can go on to the next iteration. The time taken by the slowest node at distance $d-1$ from $u$ is distributed as $max \{ X_1,...,X_{(n-\frac{n}{d})}\}$. In general, the time taken by the slowest node at distance $l$ from $u$ is distributed as $max \{ X_1,...,X_{\frac{n}{d}(d-l)}\}$. So the time $u$ have to wait to start iteration $t$ is distributed as 

$$Z = \sum_{l=0}^{d-1} max \{X_1,...,X_{\frac{n}{d}(d-l)}\}$$

$$\mathbb{E}[Z] = \frac{1}{\lambda} \sum_{j=0}^{d-1} \sum_{i=1}^{\frac{n}{d}(d-j)} \frac{1}{i}$$

Clearly this situation is very rare, but it gives the most pessimistic lower bound of the time a node should wait in order to perform one iteration.

**In the following tests the _almost_-lower bound will be used (and not the latter).**
